In [247]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [248]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [249]:
df = pd.read_csv(f'artifacts/raw_data/customer_churn_removed_col.csv')

In [250]:
df.isnull().sum()

Age                           0
Gender                        0
Location                      0
Subscription_Length_Months    0
Monthly_Bill                  0
Total_Usage_GB                0
Churn                         0
dtype: int64

## Transformation

In [251]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

In [252]:
X = df.drop(columns='Churn')
y = df['Churn']

In [253]:
cat_cols =[feature for feature in df.columns if df[feature].dtype == 'O']
num_cols =[feature for feature in df.columns if df[feature].dtype != 'O']
print(cat_cols)
print(num_cols)

['Location']
['Age', 'Gender', 'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB', 'Churn']


In [254]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder,
                                   MinMaxScaler)
import numpy as np
from sklearn.model_selection import train_test_split

In [255]:
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.30, random_state=42)

In [256]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

In [257]:
X_train

,Age,Gender,Location,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB
76513,25,0,Los Angeles,13,98.13,468
60406,54,1,Houston,2,71.90,383
27322,32,0,Chicago,20,65.65,427
53699,61,1,Houston,12,83.20,64
65412,70,1,New York,5,41.93,99
...,...,...,...,...,...,...
6265,35,0,Miami,21,67.33,235
54886,56,0,Chicago,13,85.40,347
76820,69,0,Houston,2,76.24,321
860,55,0,Chicago,12,89.19,315


In [258]:
preprocessing = ColumnTransformer(transformers=[
    ('OHE',OneHotEncoder(drop='first',sparse=False,dtype=np.int64),['Location']),
    # ('scaling',MinMaxScaler(),['Age', 'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB'])
],remainder='passthrough')

In [259]:
X_train= preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

In [260]:
from sklearn.linear_model import LogisticRegression


logst = LogisticRegression()
logst.fit(X_train,y_train)

LogisticRegression()

In [261]:
def evaluate(true,pred):
    
    cm = confusion_matrix(true, pred)
    accuracy = accuracy_score(true, pred)
    recall = recall_score(true, pred)
    precision = precision_score(true, pred)
    
    report = classification_report(true, pred)

    evaluation_report = {
        'confusion_matrix': cm,
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'classification_report': report
    }
    print('confusion metrics: \n',cm)
    print('accuracy: \n',accuracy)
    print('precision: \n',precision)
    print('recall: \n',recall)
    print('classification_report: \n',report)

In [263]:
pred = logst.predict(X_test)
evaluate(true=y_test,pred=pred)

## model

In [262]:
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, classification_report

In [232]:
rfc = RandomForestClassifier(n_estimators=200,oob_score=True)

In [233]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, oob_score=True)

In [234]:
def evaluate(true,pred):
    
    cm = confusion_matrix(true, pred)
    accuracy = accuracy_score(true, pred)
    recall = recall_score(true, pred)
    precision = precision_score(true, pred)
    
    report = classification_report(true, pred)

    evaluation_report = {
        'confusion_matrix': cm,
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'classification_report': report
    }
    print('confusion metrics: \n',cm)
    print('accuracy: \n',accuracy)
    print('precision: \n',precision)
    print('recall: \n',recall)
    print('classification_report: \n',report)
    
    # return evaluation_report


    

In [235]:
y_pred = rfc.predict(X_test)

In [236]:
from pprint import pprint

In [237]:
evaluate(true=y_test,pred=y_pred)

confusion metrics: 
 [[7738 7414]
 [7664 7184]]
accuracy: 
 0.4974
precision: 
 0.4921222085217153
recall: 
 0.4838362068965517
classification_report: 
               precision    recall  f1-score   support

           0       0.50      0.51      0.51     15152
           1       0.49      0.48      0.49     14848

    accuracy                           0.50     30000
   macro avg       0.50      0.50      0.50     30000
weighted avg       0.50      0.50      0.50     30000



## training all aglorithms

In [238]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

In [239]:
def evaluate(true,pred):
    accuracy = accuracy_score(true, pred)
    recall = recall_score(true, pred)
    precision = precision_score(true, pred)
    return accuracy , recall , precision


In [240]:
params = {
    "Gradient Boosting Classifier": {
        'loss': ['deviance', 'exponential'],  # Classification loss functions
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        'criterion': ['friedman_mse'],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
        'max_features': ['auto', 'sqrt', 'log2']
    },
    "XGBoost Classifier": {
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
        'max_depth': [3, 4, 5, 6, 7, 8],
        'min_child_weight': [1, 2, 3, 4]
    },
    "CatBoost Classifier": {
        'depth': [6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [30, 50, 100],
        'loss_function': ['Logloss', 'CrossEntropy'],
        'eval_metric': ['Logloss', 'AUC'],
    },
    "AdaBoost Classifier": {
        'learning_rate': [0.1, 0.01, 0.5, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'algorithm': ['SAMME', 'SAMME.R'],
    }
}

In [241]:
models = {
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "XGBoost Classifier": XGBClassifier(),
    "CatBoost Classifier": CatBoostClassifier(),
    "AdaBoost Classifier": AdaBoostClassifier(),
}


In [242]:

class ModelTraining:
    def __init__(self):
        self.model_list = []
        self.accuracy_list = []
        self.best_params_dict = {}

    def train_model(self, models, params, X_train, y_train, X_test, y_test):
        for i, model_name in enumerate(models.keys()):
            model = models[model_name]
            param = params[model_name]

            grid_search_cv = RandomizedSearchCV(estimator=model, param_distributions=param, cv=5)
            grid_search_cv.fit(X_train, y_train)

            best_params = grid_search_cv.best_params_
            model.set_params(**best_params)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            accuracy, recall, precision = evaluate(true=y_test, pred=y_pred)

            print(model_name)
            self.model_list.append(model_name)

            print("- precision : {:.4f}".format(precision))
            print("- recall : {:.4f}".format(recall))
            print("- accuracy : {:.4f}".format(accuracy))
            print('\n')

            self.best_params_dict[model_name] = best_params

            self.accuracy_list.append(accuracy)

    def return_accuracy_list(self):
        return self.accuracy_list
    
    def best_params(self):
        return self.best_params_dict


In [243]:
train_model = ModelTraining()

In [244]:
train_model.train_model(models=models,params=params,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)

Gradient Boosting Classifier
- precision : 0.5056
- recall : 0.0705
- accuracy : 0.5058


XGBoost Classifier
- precision : 0.4991
- recall : 0.4273
- accuracy : 0.5043


0:	learn: 0.6930922	total: 22ms	remaining: 1.08s
1:	learn: 0.6930002	total: 31.4ms	remaining: 753ms
2:	learn: 0.6929053	total: 41.7ms	remaining: 654ms
3:	learn: 0.6928449	total: 52.9ms	remaining: 608ms
4:	learn: 0.6927656	total: 61.9ms	remaining: 557ms
5:	learn: 0.6927013	total: 72.3ms	remaining: 530ms
6:	learn: 0.6926154	total: 80.9ms	remaining: 497ms
7:	learn: 0.6925162	total: 90.8ms	remaining: 477ms
8:	learn: 0.6924213	total: 101ms	remaining: 461ms
9:	learn: 0.6923345	total: 110ms	remaining: 438ms
10:	learn: 0.6922685	total: 120ms	remaining: 424ms
11:	learn: 0.6921983	total: 127ms	remaining: 402ms
12:	learn: 0.6920916	total: 138ms	remaining: 393ms
13:	learn: 0.6920572	total: 148ms	remaining: 381ms
14:	learn: 0.6919696	total: 157ms	remaining: 366ms
15:	learn: 0.6918556	total: 167ms	remaining: 356ms
16:	learn: 0.69177

In [245]:
train_model.best_params_dict

{'Gradient Boosting Classifier': {'subsample': 0.9,
  'n_estimators': 16,
  'max_features': 'log2',
  'loss': 'exponential',
  'learning_rate': 0.01,
  'criterion': 'friedman_mse'},
 'XGBoost Classifier': {'subsample': 0.8,
  'n_estimators': 16,
  'min_child_weight': 1,
  'max_depth': 6,
  'learning_rate': 0.05},
 'CatBoost Classifier': {'loss_function': 'CrossEntropy',
  'learning_rate': 0.01,
  'iterations': 100,
  'eval_metric': 'Logloss',
  'depth': 10},
 'AdaBoost Classifier': {'n_estimators': 256,
  'learning_rate': 0.001,
  'algorithm': 'SAMME.R'}}

In [246]:
model_accuracy=pd.DataFrame(list(zip(list(models.keys()),train_model.accuracy_list)),columns=['model','accuracy'])
model_accuracy

,model,accuracy
0,Gradient Boosting Classifier,0.505833
1,XGBoost Classifier,0.504333
2,CatBoost Classifier,0.498500
3,AdaBoost Classifier,0.503067
